<a href="https://colab.research.google.com/github/galymzhan-zharas/Voice-assistant/blob/main/Voice_assisstant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# write LLM text generation
# write a TTS pipeline using OpenAI API
  # replace it with a different, free model later (see how it's done in ASR tutorial in HG face)
# if all works connect them through langchain
  # ask claude and chatgpt for langchain tutorial
  # try connecting it myself
  # send the code to gpt to ask if all good
  # modularize the code (define functions, etc)
# write a GUI

In [10]:
!git config --global user.name "galymzhan-zharas"
!git config --global user.email "galymzhanzharas01@gmail.com"

In [11]:
!git clone https://github.com/galymzhan-zharas/Voice-assistant.git code

Cloning into 'code'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (6/6), 528.39 KiB | 13.55 MiB/s, done.


In [12]:
%cd code

/content/code


In [29]:
if not os.path.exists('/content/audio'):
    os.makedirs('/content/audio')

# Pinecone setup


In [17]:
!pip install -qU pinecone python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 8.4 MB/s eta 0:00:00


In [18]:
from google.colab import userdata
from pinecone import Pinecone
from langchain.text_splitter import RecursiveCharacterTextSplitter
from docx import Document
import os


In [20]:
# check if a directory called 'documents' exists
if not os.path.exists('/content/documents'):
    # create a directory called 'documents'
    os.makedirs('/content/documents')

In [19]:
# Set your API key and environment
pc = Pinecone(api_key=userdata.get('pinecone-api-key'))
index_name = 'voice-assistant'

if not pc.has_index(index_name):
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed={
            "model":"llama-text-embed-v2",
            "field_map":{"text": "chunk_text"}
        }
    )

In [ ]:
# this splitter can be used for paragraph splitting

# get the chunks out of documents
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2500,       # ~600 tokens
    chunk_overlap=300,     # overlap to retain context
    separators=["\n\n", "\n", ".", " ", ""]
)

In [ ]:
# here, I am chunking 3 documents and storing each chunk in the list.
# each chunk is represted as a dictionary that contains the text itself as well as supporting meta-data
records = []
doc_num = 0
for file_name in os.listdir('/content/documents'):
  doc_num +=1
  # ------- for reading text data -------- #

  # with open(os.path.join('/content/documents', file_name), 'r') as f:
  #   text = f.read()
  #   chunks = text_splitter.split_text(text)
  #   for chunk_num, chunk in enumerate(chunks,start=1):

  #     record = {
  #         "_id": f"document{doc_num}#chunk{chunk_num}", # it helps to update or delete specific chunk
  #         'chunk_text': chunk,
  #         'document_id':f"document{doc_num}",
  #         'document_title': file_name,
  #         'chunk_num': chunk_num
  #     }
  #     records.append(record)

  # ------- for reading docx  -------- #
  document = Document(os.path.join('/content/documents', file_name))
  full_text = []
  for para in document.paragraphs:
    full_text.append(para.text)
  text = '\n'.join(full_text)

  chunks = text_splitter.split_text(text)
  for chunk_num, chunk in enumerate(chunks,start=1):
    record = {
        "_id": f"document{doc_num}#chunk{chunk_num}", # it helps to update or delete specific chunk
        'chunk_text': chunk,
        'document_id':f"document{doc_num}",
        'document_title': file_name,
        'chunk_num': chunk_num
    }
    records.append(record)

In [32]:
dense_index = pc.Index(index_name) # just an API for the created index
dense_index.upsert_records("articles", records) # they are upserted into the namespace 'narratives'

In [ ]:
# view how many indeces you've upserted
stats = dense_index.describe_index_stats()
print(stats)

{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'articles': {'vector_count': 12},
                'narratives': {'vector_count': 6}},
 'total_vector_count': 18,
 'vector_type': 'dense'}


# LLM text generation

In [ ]:
# I can use stateless Chatcompletion first. But, then move on to an assistant. Whenever, a user makes a call, a new thread get's created
# I can wait until LLM sends the data. But, then I can use streaming text generation and convert it to speech as it's generated
# I can see the real time trancription by OpenAI


In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# from langchain import PromptTemplate


In [ ]:
# vicuna_template = PromptTemplate.from_template(
#     "USER: {question}\n"
#     "CONTEXT:\n{context}\n\n"
#     "ASSISTANT:"
# )

In [ ]:
# prompt = vicuna_template.format(
#     context=your_context,
#     question=your_question
# )
# prompt

StringPromptValue(text="<s>[INST] <<SYS>>\nYou are a helpful assistant. Answer based solely on the context below.\nIf the answer is not present, say “I don’t have enough information.”\n<</SYS>>\n\nContext:\nNoctiserpens aquavarian: The Shape-Shifting Evolutionary Lizard\nNoctiserpens aquavarian is a newly discovered, semi-mythical reptilian species exhibiting remarkable adaptive and morphogenetic abilities. First documented in 2023 in the mangrove-laced coastlines of southeastern Papua New Guinea, this unique lizard is known for its unprecedented capacity to transform into a snake-like form during the night and is showing early-stage biological adaptations toward becoming a fully aquatic marine reptile. Biologists consider it a living example of evolution in motion.\n\nTaxonomy and Classification\n\t•\tKingdom: Animalia\n\t•\tPhylum: Chordata\n\t•\tClass: Reptilia\n\t•\tOrder: Squamata\n\t•\tFamily: Noctisauridae (proposed)\n\t•\tGenus: Noctiserpens\n\t•\tSpecies: N. aquavarian\n\t•\tC

In [ ]:
# def load_chat_model(model_name: str, device: int = 0, dtype=torch.float16):
#     tokenizer = AutoTokenizer.from_pretrained(model_name)
#     model     = AutoModelForCausalLM.from_pretrained(
#         model_name,
#         torch_dtype=dtype
#         )
#     return pipeline(
#         "text-generation",
#         model=model,
#         tokenizer=tokenizer,
#         device=device,
#     )

In [ ]:
# # simply try openai's chat completion model
# chat = load_chat_model("lmsys/vicuna-7b-v1.5")
# response = chat(
#     prompt,
#     max_new_tokens=512,
#     do_sample=True,
#     temperature=0.7
# )
# print(response[0]["generated_text"])

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Device set to use cuda:0


OutOfMemoryError: CUDA out of memory. Tried to allocate 250.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 56.12 MiB is free. Process 15465 has 14.68 GiB memory in use. Of the allocated memory 14.53 GiB is allocated by PyTorch, and 21.32 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
# model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
# inputs = tokenizer(prompt, return_tensors="pt") # tokenizes the text, transforms into numbers, and returns tensors
# out = model.generate(**inputs, max_new_tokens=1024) # takes in the token ids and generates numbers
# answer = tokenizer.decode(out[0], skip_special_tokens=True) # takes in the token ids and generates text
# answer

# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
# inputs = tokenizer(prompt, return_tensors="pt") # tokenizes the text, transforms into numbers, and returns tensors
# out = model.generate(**inputs, max_new_tokens=1024) # takes in the token ids and generates numbers
# answer = tokenizer.decode(out[0], skip_special_tokens=True) # takes in the token ids and generates text
# answer

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2.
403 Client Error. (Request ID: Root=1-685aa5a1-2b653e216c160a864c6e5150;dc4762e7-5c55-4f2c-8aae-54db2d2c5bda)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.2 is restricted and you are not in the authorized list. Visit https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2 to ask for access.

# Main function

In [21]:
!pip install -qU transformers

In [22]:
!pip install -q openai pygame soundfile

In [23]:
from transformers import pipeline
import torch
from openai import OpenAI
import pygame
import io
import soundfile as sf
import tempfile
import librosa
from IPython.display import Audio, display

pygame 2.6.1 (SDL 2.28.4, Python 3.11.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [59]:
if torch.cuda.is_available():
    device = "cuda:0"
    torch_dtype = torch.float16
else:
    device = "cpu"
    torch_dtype = torch.float32

#most likely the audio files will be longer than 30s, set chunking to true
whisper_med = pipeline("automatic-speech-recognition", model="openai/whisper-medium", device=device, torch_dtype=torch_dtype, chunk_length_s=30)


Device set to use cuda:0


In [65]:
def transcribe(audio, pipe):
  # print(pipe(audio)["text"])
  return pipe(audio)["text"]

In [78]:
def get_context(query, index, namespace, threshold=0.3):
  # print('query', query)
  results = index.search(
    namespace=namespace,
    query={
        "top_k": 10,
        "inputs": {
            'text': query
        }
    },
    rerank={
      "model": "bge-reranker-v2-m3",
      "top_n": 10,
      "rank_fields": ["chunk_text"]
    }
  )
  # print(results)
  results_filtered = [hit for hit in results['result']['hits'] if hit['_score'] > threshold]
  results_filtered.sort(key=lambda x: (int(x['fields']['document_id'][-1]), x['fields']['chunk_num']))

  chunk_texts = [chunk['fields']['chunk_text'] for chunk in results_filtered]
  context = '\n'.join(chunk_texts)
  print(f"{context}")
  return context

In [42]:
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

def generate_answer(query, context, model='gpt-3.5-turbo', temperature=0.5, max_tokens=512):
  messages = [
    {"role": "system", "content": "You are a helpful assistant. Answer based only on the provided context. If unsure, say 'I don't know.'"},
    {"role": "user",   "content": f"Context:\n{context}\n\nQuestion:\n{query}"}
  ]
  response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=temperature,
    max_tokens=512
  )

  return response.choices[0].message.content


In [66]:
# ["alloy", "echo", "fable", "onyx", "nova", "shimmer"]
def TTS(text):
    response = client.audio.speech.create(
        model="tts-1",
        voice="nova",
        input=text
    )
    display(Audio(response.content, autoplay=True))
    print("✅ Speech generation completed")

In [79]:
def main(audio_file):
  audio, _ = librosa.load(audio_file, sr=16000)
  text = transcribe(audio, whisper_med)
  context = get_context(text, dense_index, 'articles')
  answer = generate_answer(text, context)
  TTS(answer)

In [80]:
main('/content/audio/Lizard appearance.wav')

/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Noctiserpens aquavarian: The Shape-Shifting Evolutionary Lizard
Noctiserpens aquavarian is a newly discovered, semi-mythical reptilian species exhibiting remarkable adaptive and morphogenetic abilities. First documented in 2023 in the mangrove-laced coastlines of southeastern Papua New Guinea, this unique lizard is known for its unprecedented capacity to transform into a snake-like form during the night and is showing early-stage biological adaptations toward becoming a fully aquatic marine reptile. Biologists consider it a living example of evolution in motion.

Taxonomy and Classification
	•	Kingdom: Animalia
	•	Phylum: Chordata
	•	Class: Reptilia
	•	Order: Squamata
	•	Family: Noctisauridae (proposed)
	•	Genus: Noctiserpens
	•	Species: N. aquavarian
	•	Common Names: Twilight Shifter, Serpent Lizard, Aqua Wyrm

Appearance
In its diurnal (daytime) form, N. aquavarian resembles a slender, arboreal lizard approximately 50–70 cm in length. Its scales are iridescent, with hues that shift f

✅ Speech generation completed
